In [1]:
from datetime import date as d
import numpy as np
import pandas as pd
import math 
import matplotlib.pyplot as plt
import datetime
import fix_yahoo_finance as fyf
import xlrd
import time

In [2]:
interval=5
to_period=d.today()
from_period=d.today() - datetime.timedelta(days=365*5)

In [3]:
ticker = input('Input a Stock/Currency Pair Name') #Type the stock name you want
stock_data=fyf.download(ticker,start=from_period,end=to_period,interval = '1d')[['Adj Close']].round(2)

Input a Stock/Currency Pair Name SQ


[*********************100%***********************]  1 of 1 downloaded


In [4]:

stock_data['Returns']=(stock_data['Adj Close']/stock_data['Adj Close'].shift(1))-1  


In [8]:

avg_simplereturns_monthly=(stock_data['Returns'].tail(21)).mean()*21
print("avg_simple_returns_monthly = %.2f%%" % (100*float(avg_returns_monthly)))
avg_simplereturns_quarterly=(stock_data['Returns'].tail(63)).mean()*63
print("avg_simple_returns_quarterly = %.2f%%" % (100*float(avg_returns_quarterly)))
avg_simplereturns_annually=(stock_data['Returns'].tail(252)).mean()*252
print("avg_simple_returns_annually = %.2f%%" % (100*float(avg_returns_annually)))

avg_simple_returns_monthly = -11.10%
avg_simple_returns_quarterly = -6.69%
avg_simple_returns_annually = 37.57%


In [9]:
stock_data['Returns']=np.log(stock_data['Adj Close']/stock_data['Adj Close'].shift(1))  
avg_logreturns_monthly=(stock_data['Returns'].tail(21)).mean()*21
print("avg_log_returns_monthly = %.2f%%" % (100*float(avg_logreturns_monthly)))
avg_logreturns_quarterly=(stock_data['Returns'].tail(63)).mean()*63
print("avg_log_returns_quarterly = %.2f%%" % (100*float(avg_logreturns_quarterly)))
avg_logreturns_annually=(stock_data['Returns'].tail(252)).mean()*252
print("avg_log_returns_annually = %.2f%%" % (100*float(avg_logreturns_annually)))

avg_lo_greturns_monthly = -11.66%
avg_log_returns_quarterly = -8.04%
avg_log_returns_annually = 21.92%


In [10]:
interval=np.arange(-0.025,0.025,0.005)
hist_data= pd.DataFrame(columns=['Bins', 'Count'])
for i in range(len(interval)):
    if i==0:
        count=stock_data[stock_data['Returns']<= interval[i+1]].shape[0]
        cols=  '<'+str ((interval[i+1]*100).round(2))+'%'
    elif i==len(interval)-1:
        count=stock_data[stock_data['Returns']> interval[i]].shape[0]
        cols=  '>'+str ((interval[i]*100).round(2))+'%'
    else:
        tr=np.logical_and(stock_data['Returns']> interval[i],stock_data['Returns']<= interval[i+1])
        count=stock_data[tr].shape[0]
        cols=  str((interval[i]*100).round(2))+'% to ' +str((interval[i+1]*100).round(2))+'%' 
    hist_data1=pd.DataFrame([[cols,count]],columns=['Bins', 'Count'])  
    hist_data=hist_data.append(hist_data1,ignore_index=True)   

In [11]:
Describe=pd.DataFrame(stock_data['Returns'].describe())
Describe.reset_index(inplace=True)
Describe.rename(columns={'index':'Description'}, inplace=True)
mean=Describe.iloc[1,1]
std=Describe.iloc[2,1]

In [ ]:
max()

In [12]:
tr=np.logical_and(stock_data['Returns']>=mean-std,stock_data['Returns']<=mean+std)
tr2=np.logical_and(stock_data['Returns']>=mean-2*std,stock_data['Returns']<=mean+2*std)
within=round(stock_data[tr].shape[0]/stock_data['Returns'].shape[0],2)
within2=round(stock_data[tr2].shape[0]/stock_data['Returns'].shape[0],2)
within=str(within*100)+'%'
within2=str(within2*100)+'%'
Describe=Describe.append(pd.DataFrame([['Within 1 Std',within]],columns=['Description','Returns']),
                         ignore_index=True)
Describe=Describe.append(pd.DataFrame([['Within 2 Std',within2]],columns=['Description','Returns']),
                         ignore_index=True)

In [13]:
intr1_upper=mean+std
intr1_lower=mean-std
intr2_upper=mean+2*std
intr2_lower=mean-2*std
intr3_upper=mean+3*std
intr3_lower=mean-3*std
stdintr=pd.DataFrame([['Upper',intr1_upper,intr2_upper,intr3_upper],
                      ['Lower',intr1_lower,intr2_lower,intr3_lower]],columns=['Std Dev',1,2,3])
    

In [14]:
from pandas import ExcelWriter

writer=pd.ExcelWriter(ticker+'_Odds & Distribution.xlsx',datetime_format='dd-mm-yyyy')
workbook=writer.book
worksheet=workbook.add_worksheet(ticker)
writer.sheets[ticker] = worksheet
stock_data.to_excel(writer, sheet_name=ticker, index= True)
hist_data.to_excel(writer,sheet_name=ticker,startrow=0, startcol=10,index=False) 
Describe.to_excel(writer,sheet_name=ticker,startrow=0, startcol=14,index=False)
stdintr.to_excel(writer,sheet_name=ticker,startrow=0, startcol=17,index=False)
percentage= workbook.add_format({'num_format':'0.00%'})
worksheet.set_column('D:D',13, percentage)
worksheet.set_column('S:U',13, percentage)
for i, col in enumerate(stock_data.columns):
    column_len = len(col)
    # set the column length
    if(i==0):
        worksheet.set_column(i+10, i+10, column_len+10)
        worksheet.set_column(i+14, i+14, column_len+10)
        worksheet.set_column(i, i, column_len+10)
    break    
    
chart=workbook.add_chart({'type':'column'})
chart.add_series({
    'values':ticker+'!$L$2:$L$11',
    'categories':"'"+ticker+"'"+'!$K$2:$K$11',
    'gap':15,
})
chart.set_size({'x_scale': 1, 'y_scale': 1.3})
chart.set_x_axis({'name':'Return','num_font':  {'rotation': 45}})
chart.set_y_axis({'name':'Count'})
chart.set_legend({'none': True})
chart.set_title({'name': 'Histogram of Returns'})
worksheet.insert_chart('K2', chart, {'x_offset': 20, 'y_offset': 250})
writer.save()